## Load the dataset


In [1]:
from VQA_Datasetv2 import VQA_Dataset, VQA_Dataset_preloaded
import clip
import torch
from torch.utils.data import DataLoader
from torch.utils.data import random_split

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
model, preprocess = clip.load("ViT-B/32", device=device)
model.to(torch.float32)
print("clip model loaded")

Using cuda device
clip model loaded


## Choose one of the 3:

### 1.Normal loading

In [ ]:
loading = 3

if loading < 3:
    dataset = VQA_Dataset()
    
if loading == 0:
    dataset.load_preprocess(preprocess, device, length=6000)
elif loading == 1:
    dataset.load(preprocess, device, length=30000)
elif loading == 2:
    dataset.load_encode(preprocess, device, model, length=100)
    dataset.save("")
elif loading == 3:
    dataset.load_saved("")

### 2.Precalc

In [2]:
dataset = VQA_Dataset_preloaded()
dataset.compute_store(preprocess, model, device, "full", length=30000)

Preprocessing Images: 100%|██████████| 30000/30000 [31:50<00:00, 15.70it/s]


### 3.Only load

In [2]:
dataset = VQA_Dataset_preloaded()
dataset.load("full", device)

### Create Dataloader

In [3]:


train_val_size = int(len(dataset)*0.8)
train_size = int(len(dataset)*0.8*0.8)
val_size = int(len(dataset)*0.8)-train_size
test_size = int(len(dataset))-train_val_size
print("Train size: ", train_size)
print("Test size: ", test_size)
print("Val size: ", val_size)
generator = torch.Generator().manual_seed(42)
train_dataset, test_dataset, val_dataset = random_split(dataset, [train_size, test_size, val_size], generator=generator)

batch_size=32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

Train size:  19200
Test size:  6000
Val size:  4800


## Simple model architectue


In [4]:
from models import VQA_Model3, VQA_Model2, VQA_Model_Precalc, VQA_Model4



## Evaluate

In [5]:
import tqdm

def evaluate(model, dataloader, device, test_size, show_progress=False):
    model.eval()
    correct = 0
    if show_progress:
        pbar = tqdm.tqdm(dataloader)
    else:
        pbar = dataloader
    for i, data in enumerate(pbar):
        image = data[0].to(device)
        answer_tokens = data[1].squeeze(0).to(device)
        if dataloader.batch_size == 1:
            correct_answer = torch.tensor([int(data[3])]).to(device)
            question_tokens = data[2].squeeze(0).to(device)
        else:
            correct_answer = torch.tensor([int(x) for x in data[3]]).to(device)
            question_tokens = data[2].squeeze(1).to(device)

        with torch.no_grad():
            similarity = model(image, question_tokens, answer_tokens)
            pred = similarity.argmax(dim=-1)
            # get sum of correct answers
            correct += (pred == correct_answer).sum().item()
    print(f"Correct: {correct}, Total: {test_size}")
    return correct/(test_size)


In [6]:
combined_model = VQA_Model_Precalc(model, device)
evaluate(combined_model, test_dataloader, device, test_size, show_progress=True)

100%|██████████| 188/188 [00:44<00:00,  4.23it/s]

Correct: 147, Total: 6000


0.0245

## Training

In [8]:
def train(model, train_dataloader, val_dataloader, device, test_size, epochs=10, patience=2, precalculated=False):
    print(device)
    model.train()
    print(model.parameters())
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    loss_fn = torch.nn.CrossEntropyLoss()

    pbar = tqdm.tqdm(range(epochs))
    prev_acc = 0.0
    best_acc = 0.0
    count = 0
    for epoch in pbar:
        for i, data in enumerate(train_dataloader):

            image = data[0].to(device)
            answer_tokens = data[1].squeeze(0).to(device)
            if train_dataloader.batch_size == 1:
                correct_answer = torch.tensor(int(data[3])).to(device)
                question_tokens = data[2].squeeze(0).to(device)
            else:
                correct_answer = torch.tensor([int(x) for x in data[3]]).to(device)
                question_tokens = data[2].squeeze(1).to(device)
            
            optimizer.zero_grad()
            similarity = model(image, question_tokens, answer_tokens)

            # transform asnwer to tensor of the same shape as similarity before only correct index
            #one_hot_encoding = torch.zeros(similarity.shape[1])
            #one_hot_encoding[correct_answer] = 1
            #one_hot_encoding = one_hot_encoding.to(device)
            

            loss = loss_fn(similarity, correct_answer)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=4.0, error_if_nonfinite=True)
            optimizer.step()
            #print("--------------------------------------------------")
        acc = evaluate(model, val_dataloader, device,test_size)
        if acc > best_acc:
            best_acc = acc
            # save model
            torch.save(model.state_dict(), "best_model.pt")
        if acc < prev_acc:
            count += 1
        else:
            count = 0
        prev_acc = acc
        pbar.set_description(f"Epoch {epoch} loss: {loss.item()}, acc: {acc}", refresh=True)
        if count == patience:
            print("early stopping")
            break 

In [9]:
import time

trained_model = VQA_Model_Precalc(model, device)
# freeze the clip model
for param in trained_model.model.parameters():
    param.requires_grad = False


evaluate(trained_model, test_dataloader, device, test_size)
train(trained_model, train_dataloader, val_dataloader, device, val_size, epochs=200)
# evaluate the model
evaluate(trained_model, test_dataloader, device, test_size)

Correct: 727, Total: 6000
cuda
<generator object Module.parameters at 0x00000153DF47E490>


  0%|          | 0/200 [00:00<?, ?it/s]

Correct: 1871, Total: 4800


Epoch 0 loss: 1.6668224334716797, acc: 0.38979166666666665:   0%|          | 1/200 [02:51<9:28:09, 171.30s/it]

Correct: 2130, Total: 4800


...

Correct: 2517, Total: 4800


Epoch 11 loss: 0.4858710765838623, acc: 0.5216666666666666:   6%|▌         | 11/200 [35:04<10:02:32, 191.28s/it]

Correct: 2504, Total: 4800
early stopping


Correct: 3181, Total: 6000


0.5301666666666667

## Save the model!

In [2]:
# save the model
import torch
torch.save(trained_model.state_dict(), "trained_model.pt")

NameError: name 'trained_model' is not defined

## Load and evaluate


In [6]:
#load and evaluate the model
from models import VQA_Model4

trained_model = VQA_Model4(model, device) # need to choose the same model
trained_model.load_state_dict(torch.load("Trained_Models/best_model-v4-60.pt"))
#print(trained_model)
evaluate(trained_model, test_dataloader, device, test_size, show_progress=True)

VQA_Model4(
  (model): CLIP(
    (visual): VisionTransformer(
      (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
      (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (transformer): Transformer(
        (resblocks): Sequential(
          (0): ResidualAttentionBlock(
            (attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
            )
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (mlp): Sequential(
              (c_fc): Linear(in_features=768, out_features=3072, bias=True)
              (gelu): QuickGELU()
              (c_proj): Linear(in_features=3072, out_features=768, bias=True)
            )
            (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          )
          (1): ResidualAttentionBlock(
            (attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizable

0.5816666666666667

In [ ]:
"""
import clip
from PIL import Image
import torch

for model_name in clip.available_models():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using {device} device and model: {model_name}")
    model, preprocess = clip.load(model_name, device=device)
    model.to(torch.float32)
    text_tokens = clip.tokenize(["question_text"]).to(device)
    image = preprocess(Image.open("CLIP.jpg")).unsqueeze(0).to(device)
    image_features = model.encode_image(image)
    text_features = model.encode_text(text_tokens)
    print(f"text_t {text_tokens.dtype}, text_f {text_features.dtype}, image_t {image.dtype}, image_f {image_features.dtype}")
    """


'\nimport clip\nfrom PIL import Image\nimport torch\n\nfor model_name in clip.available_models():\n    device = "cuda" if torch.cuda.is_available() else "cpu"\n    print(f"Using {device} device and model: {model_name}")\n    model, preprocess = clip.load(model_name, device=device)\n    model.to(torch.float32)\n    text_tokens = clip.tokenize(["question_text"]).to(device)\n    image = preprocess(Image.open("CLIP.jpg")).unsqueeze(0).to(device)\n    image_features = model.encode_image(image)\n    text_features = model.encode_text(text_tokens)\n    print(f"text_t {text_tokens.dtype}, text_f {text_features.dtype}, image_t {image.dtype}, image_f {image_features.dtype}")\n    '